In [154]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [155]:
tf.__version__

'2.16.1'

We are determining if given some charecteristics, will a customer exit or not from a bank.

In [156]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [172]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [157]:
print(data.groupby('Geography').size())
print(data.groupby('Gender').size())
print(data.groupby('HasCrCard').size())
print(data.groupby('NumOfProducts').size())

Geography
France     5014
Germany    2509
Spain      2477
dtype: int64
Gender
Female    4543
Male      5457
dtype: int64
HasCrCard
0    2945
1    7055
dtype: int64
NumOfProducts
1    5084
2    4590
3     266
4      60
dtype: int64


In [158]:
x = data.iloc[:,3:-1].values # ignoring customer id, row nums and surname columns
y = data.iloc[:,-1].values
x.shape, y.shape
x

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [159]:
# for gender column
encoder = LabelEncoder()
x[:,2] = encoder.fit_transform(x[:,2])
x

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [160]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [161]:
x

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [162]:
X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=0.2, random_state=0)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((8000, 12), (2000, 12), (8000,), (2000,))

In [163]:
X_train

array([[0.0, 0.0, 1.0, ..., 1, 0, 163830.64],
       [0.0, 1.0, 0.0, ..., 1, 1, 57098.0],
       [1.0, 0.0, 0.0, ..., 1, 0, 185630.76],
       ...,
       [1.0, 0.0, 0.0, ..., 1, 0, 181429.87],
       [0.0, 0.0, 1.0, ..., 1, 1, 148750.16],
       [0.0, 1.0, 0.0, ..., 1, 0, 118855.26]], dtype=object)

In [164]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train, X_test

(array([[-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
         -1.03227043,  1.10643166],
        [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
          0.9687384 , -0.74866447],
        [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
         -1.03227043,  1.48533467],
        ...,
        [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
         -1.03227043,  1.41231994],
        [-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
          0.9687384 ,  0.84432121],
        [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
         -1.03227043,  0.32472465]]),
 array([[-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
          0.9687384 ,  1.61085707],
        [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
         -1.03227043,  0.49587037],
        [-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
          0.9687384 , -0.42478674],
        ...,
        [-1.01460667, -0.5698444 ,  1.74309049, ...,  

In [165]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [166]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [167]:
ann.fit(X_train, Y_train, batch_size = 32, epochs = 100)

Epoch 1/100


250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 273us/step - accuracy: 0.6744 - loss: 0.6380
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 225us/step - accuracy: 0.7956 - loss: 0.4927
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step - accuracy: 0.8022 - loss: 0.4451
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 222us/step - accuracy: 0.8261 - loss: 0.4092
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 221us/step - accuracy: 0.8320 - loss: 0.3985
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 217us/step - accuracy: 0.8328 - loss: 0.3972
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 221us/step - accuracy: 0.8412 - loss: 0.3779
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 218us/step - accuracy: 0.8477 - loss: 0.3614
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 220us/step - accuracy: 0.8547 - loss: 0.3643
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 234us/step - accuracy: 0.8534 - loss: 0.3535
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 222us/step - accuracy: 0.8528 - loss: 0.3484
Epoch 12/100
250/250 ━━━━━━━━━━

In [168]:
ann.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (32, 6)                │            78 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (32, 6)                │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (32, 1)                │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 383 (1.50 KB)

 Trainable params: 127 (508.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 256 (1.00 KB)

In [169]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[False]]


In [170]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), Y_test.reshape(len(Y_test),1)),1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 208us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [171]:
cm = confusion_matrix(Y_test, y_pred)
print(cm)
accuracy_score(Y_test, y_pred) * 100

[[1503   92]
 [ 190  215]]


85.9